Travail temporaire : 

- La façon d'extraire les lignes remplacera celle de xml_to_parquet (pour extraire le bloc budget)
- la boucle de recherche sera faite dans chaque fichier pour les ouvrir une fois au lieu de 31. 


In [ ]:
import glob 
import os 
import gzip
import xmltodict 
import pandas as pd 
import polars as pl 

In [ ]:
LIST_ANNEXES = ['DATA_AMORTISSEMENT_METHODE', 'DATA_APCP', 'DATA_AUTRE_ENGAGEMENT',
        'DATA_CHARGE', 'DATA_CONCOURS', 'DATA_CONSOLIDATION', 'DATA_CONTRAT_COUV', 
        "DATA_CONTRAT_COUV_REFERENCE", "DATA_CREDIT_BAIL", "DATA_DETTE", "DATA_EMPRUNT", 
        "DATA_ETAB_SERVICE", "DATA_FISCALITE", "DATA_FOND_AIDES_ECO", "DATA_FOND_COMM_HEBERGEMENT", 
        "DATA_FOND_EUROPEEN", "DATA_FOND_EUROPEEN_PROGRAMMATION", "DATA_FORMATION", 
        "DATA_FORMATION_PRO_JEUNES", "DATA_MEMBRESASA", "DATA_ORGANISME_ENG", 
        "DATA_ORGANISME_GROUP", "DATA_PATRIMOINE", "DATA_PERSONNEL", "DATA_PERSONNEL_SOLDE", 
        "DATA_PPP", "DATA_PRET", "DATA_PROVISION", "DATA_RECETTE_AFFECTEE", 
        "DATA_SERVICE_FERROVIAIRE_BUD", "DATA_SERVICE_FERROVIAIRE_PATRIM", 
        "DATA_SERVICE_FERROVIAIRE_TER", "DATA_SIGNATAIRE", "DATA_SIGNATURE", 
        "DATA_SOMMAIRE", "DATA_TIERS", "DATA_TRESORERIE", "DATA_VENTILATION", "DATA_FLUX_CROISES"]

In [ ]:
def parse_fichier(chemin) : 
 '''Ouvre et parse le fichier gzip'''
 with gzip.open(chemin, 'rb') as fichier_ouvert : 
  fichier_xml_gzip = fichier_ouvert.read()
  fichier_xml = fichier_xml_gzip.decode('latin-1')
  fichier_dict = xmltodict.parse(fichier_xml)
 return fichier_dict

def _isolement_id(fichier) : 
 '''Extrait l'id du nom du fichier pour la liste comprehension de securité

 ATTENTION, le premier split / va changer si on l'applique sur du minio '''
 val_id_fichier_source = fichier.split("/")[-1].split('.')[0]
 if '-' in val_id_fichier_source : 
  val_id_fichier = val_id_fichier_source.split('-')[1]
 else : 
  val_id_fichier= val_id_fichier_source
 return val_id_fichier


 
def extraction_annexe(chemin_annexe, dict_metadonnees) : 
 liste_annexe = []
 for row in chemin_annexe : 
  liste_par_ligne = {}
  for a, b in row.items() : 
   liste_par_ligne.update({a : b.get('@V')})
   liste_par_ligne.update(dict_metadonnees)
  liste_annexe.append(liste_par_ligne)
 return liste_annexe 

def recherche_dans_parse_depuis_data(dict_parse, data_annexe) : 
 enfant = data_annexe.split('DATA_')[1]
 dict_annexe = dict_parse['DocumentBudgetaire']['Budget']['Annexes'][data_annexe][enfant]
 return dict_annexe 

def recherche_annexe(fichier_parse, annexe_voulue) : 
 ''' A partir d'un nom d'annexe, comme emprunt ou concours, 
 la fonction va simplement produire le chemin dans le dict du xml pour atteindre les données voulues '''

 annexe_maj = annexe_voulue.upper()
 dict_annexe = fichier_parse['DocumentBudgetaire']['Budget']['Annexes'][f'DATA_{annexe_maj}'][annexe_maj]
 return dict_annexe 



In [ ]:
def xml_to_annexe_specifique_stable(chemin_voulu, annexe):
    chemin_via_glob = glob.glob(os.path.join(chemin_voulu, "*.gz"))
    liste_vide = []

    nb_nat09 = 0
    valide = 0
    non_valide = 0
    for fichier in chemin_via_glob:
        try:
            fichier_parse = parse_fichier(fichier)
            id_fichier = _isolement_id(fichier)

            IdColl = fichier_parse['DocumentBudgetaire']['EnTeteDocBudgetaire']['IdColl']['@V']
            Exer = fichier_parse['DocumentBudgetaire']['Budget']['BlocBudget']['Exer']['@V']
            NatDec = fichier_parse['DocumentBudgetaire']['Budget']['BlocBudget']['NatDec']['@V']
            
            if NatDec == '09' :
                nb_nat09 += 1
                if Exer != '2020' : 
                    pass
                else : 
                 valide += 1
                 dict_metadonnees = {'Id_Fichier': id_fichier, 'Exer': Exer, 'NatDec': NatDec}
                 dict_annexe = recherche_dans_parse_depuis_data(fichier_parse, annexe)
                 recup_annexe = extraction_annexe(dict_annexe, dict_metadonnees)
                 df_annexe = pl.DataFrame(recup_annexe)
                 liste_vide.append(df_annexe)
            else:
                non_valide +=1
                pass
                
        except Exception as e:
            pass

    print(valide, non_valide)
    df_global = pl.concat(liste_vide, how = 'diagonal').lazy()
    df_global.sink_parquet(f'../../../data/annexe/{annexe.upper()}_2020.parquet')


In [ ]:
for i in LIST_ANNEXES[0:4] : 
    xml_to_annexe_specifique_stable('../../../data/todo_xml/', i)
    print(i, "terminé")

temps : long. 